# Redes Neuronales Recurrentes

En este tutorial vamos a ver como entrenar una red neuronal recurrente modelando lenguaje. La objetivo es obtener un modelo probabilista en que se le asignan probabilidades a oraciones.

Para esto vamos a utilizar los datos de Penn Tree Bank (PTB).

## Archivos para este tutorial

Este tutorial referencia los siguientes archivos de **models/rnn/ptb**:

* ptb_word_lm.py : el cual es para entrenar el modelo del lenguaje.
* reader.py : Se utiliza para leer los datos.

## Descargar y preparar los datos

Los datos que ocupamos se encuentran en esta direccion: http://www.fit.vutbr.cz/~imikolov/rnnl/simple-examples.tgz

Los datos ya estan procesados y contiene 10000 diferentes palabras.

# El Modelo

## LSTM

La base de este modelo consiste en la celda LSTM que procesa una palabra a la vez y calcula la probabilidad de la continuacion de esa oracion.

El estado de la memoria empieza en cero y se va actualizando conforme se lee cada palabra.

El pseudocodigo es el siguiente:

In [ ]:
import tensorflow as tf


lstm = rnn_cell.BasicLSTMCell(lstm_size)
# Initial state of the LSTM memory.
state = tf.zeros([batch_size, lstm.state_size])

loss = 0.0
for current_batch_of_words in words_in_dataset:
    # The value of state is updated after processing each batch of words.
    output, state = lstm(current_batch_of_words, state)

    # The LSTM output can be used to make next word predictions
    logits = tf.matmul(output, softmax_w) + softmax_b
    probabilities = tf.nn.softmax(logits)
    loss += loss_function(probabilities, target_words)


# Backpropagation Truncada

En orden de hacel el proceso de aprendizaje trazeable, es comun truncar la gradiente del backpropagation a un numero fijo (**num_steps**).

A continuacion una version simplificada de la creacion de grafo para una backpropagation truncada.

In [ ]:
# Placeholder for the inputs in a given iteration.
words = tf.placeholder(tf.int32, [batch_size, num_steps])

lstm = rnn_cell.BasicLSTMCell(lstm_size)
# Initial state of the LSTM memory.
initial_state = state = tf.zeros([batch_size, lstm.state_size])

for i in range(len(num_steps)):
    # The value of state is updated after processing each batch of words.
    output, state = lstm(words[:, i], state)

    # The rest of the code.
    # ...

final_state = state

Y asi es como se implementara una iteracion sobre todo los datos.

In [ ]:
# A numpy array holding the state of LSTM after each batch of words.
numpy_state = initial_state.eval()
total_loss = 0.0
for current_batch_of_words in words_in_dataset:
    numpy_state, current_loss = session.run([final_state, loss],
        # Initialize the LSTM state from the previous iteration.
        feed_dict={initial_state: numpy_state, words: current_batch_of_words})
    total_loss += current_loss

## Imputs

La id de las palabras se les dara una densa representacion antes de darsela al LSTM. Esto permite al modelo a representar eficientemente el conocimiento de las palabras. Y tambien es facil de escribir:

In [ ]:
# embedding_matrix is a tensor of shape [vocabulary_size, embedding size]
word_embeddings = tf.nn.embedding_lookup(embedding_matrix, word_ids)

La matriz va a ser inicializada de forma aleatoria y el modelo va a aprender a difereciar el significado de las palabras solo viendo la informacion.

## Funcion de perdida

Queremos minimizar la probabilidad promedio negativa de la palabra objetivo.
Esto no es muy dificil de implementar pero la funcion **sequence_loss_by_example** esta disponible, asi que podemos utilizarla.

## Utilizando multiples LSTMs

Para dar a nuestro modelo mas poder expresivo, podemos agregar multiples capas de LSTM a los datos procesados.

LLamamos a esto **MultiRNNCell**.

In [ ]:
lstm = rnn_cell.BasicLSTMCell(lstm_size)
stacked_lstm = rnn_cell.MultiRNNCell([lstm] * number_of_layers)

initial_state = state = stacked_lstm.zero_state(batch_size, tf.float32)
for i in range(len(num_steps)):
    # The value of state is updated after processing each batch of words.
    output, state = stacked_lstm(words[:, i], state)

    # The rest of the code.
    # ...

final_state = state

# Correr el Codigo

Asumo que ya has instalado via pip la reposteria de tensorflow. 

Despues escribe esto LdC: **cd tensorflow/models/rnn/ptb python ptb_word_lm --data_path=/tmp/simple-examples/data/ --model small**

Hay 3 configuraciones soportadas por el modelo, en este tutorial usaremos el small. la diferencia es el tamaño y los parametros que se usan para entrenar.

Entre mas grande el modelo mejor deberian de ser los resultados.